# SQL query from table names - Continued

In [11]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [14]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [4]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [17]:
#new
context_user = context.copy()
print(return_CCRMSQL("""show all users""", context_user))

```sql
SELECT * FROM users;
```


In [30]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("show all users", old_context_user))

This is your SQL:
```sql
SELECT * FROM employees;
```
This SQL command retrieves all users from the "employees" table.


In [27]:
#new
print(return_CCRMSQL("find customers with highest bank accounts", context_user))

```sql
SELECT customer_id, customer_name, bank_account_balance
FROM customers
ORDER BY bank_account_balance DESC
LIMIT 5;
```


In [29]:
#old
print(return_CCRMSQL("find customers with highest bank accounts", old_context_user))

This is your SQL:
```sql
SELECT ID_usr, MAX(salary) AS highest_salary
FROM salary
GROUP BY ID_usr
ORDER BY highest_salary DESC;
```

This SQL query selects the `ID_usr` and the maximum `salary` from the `salary` table for each user. It groups the results by `ID_usr`, orders them by the highest salary in descending order, and returns the customers with the highest bank accounts.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [34]:
#new
context_user = context.copy()
print(return_CCRMSQL("""show all users my multiple types bank accounts in a database """, context_user))

```sql
SELECT u.user_id, u.user_name, b.account_type
FROM users u
JOIN bank_accounts b ON u.user_id = b.user_id
GROUP BY u.user_id, b.account_type
HAVING COUNT(DISTINCT b.account_type) > 1;
```


In [33]:
#old
print(return_CCRMSQL("""show all users my multiple types bank accounts in a database """, old_context_user))

This is your SQL:
```sql
SELECT * 
FROM bank_accounts
WHERE user_id IN (
    SELECT user_id
    FROM bank_accounts
    GROUP BY user_id
    HAVING COUNT(DISTINCT account_type) > 1
);
```

This SQL query retrieves all users who have multiple types of bank accounts in the database. It uses a subquery to first identify the user IDs that have more than one distinct account type, and then selects all the information for those users from the bank_accounts table.


In [35]:
#new
context_user = context.copy()
print(return_CCRMSQL("""Assume you're given a table Twitter tweet data, write a query to obtain a histogram of tweets posted per user in 2022. """, context_user))

```sql
SELECT user_id, COUNT(*) AS tweet_count
FROM Twitter_tweets
WHERE YEAR(posted_at) = 2022
GROUP BY user_id
ORDER BY tweet_count DESC;
```


In [36]:
#old
print(return_CCRMSQL("""Assume you're given a table Twitter tweet data, write a query to obtain a histogram of tweets posted per user in 2022. """, old_context_user))

This is your SQL:
```sql
SELECT user_id, COUNT(*) AS tweet_count
FROM Twitter_tweets
WHERE YEAR(tweet_date) = 2022
GROUP BY user_id;
```

This SQL query selects the user_id and counts the number of tweets posted by each user in the year 2022 from the Twitter_tweets table. It then groups the results by user_id to create a histogram of tweets posted per user in 2022.


In [41]:
#old
print(return_CCRMSQL("""Assume you have a table of user posts on a forum with a posts table (columns: post_id, user_id, post_date, views) and a comments table (columns: comment_id, post_id, user_id, comment_date). Write a query to identify the top 10 most influential users, defined as those whose posts have the highest ratio of comments to views, but only considering posts made in the last 6 months """, old_context_user))

This is your SQL:

```sql
SELECT user_id, SUM(comment_count) / SUM(views) AS comment_to_views_ratio
FROM (
    SELECT p.user_id, COUNT(c.comment_id) AS comment_count, p.views
    FROM posts p
    JOIN comments c ON p.post_id = c.post_id
    WHERE p.post_date >= DATE_SUB(NOW(), INTERVAL 6 MONTH)
    GROUP BY p.post_id
) AS subquery
GROUP BY user_id
ORDER BY comment_to_views_ratio DESC
LIMIT 10;
```

This query first joins the posts and comments tables to calculate the number of comments and views for each post made in the last 6 months. Then, it calculates the ratio of comments to views for each user and selects the top 10 users with the highest ratio, making them the most influential users.


In [42]:
#new
context_user = context.copy()
print(return_CCRMSQL("""Assume you have a table of user posts on a forum with a posts table (columns: post_id, user_id, post_date, views) and a comments table (columns: comment_id, post_id, user_id, comment_date). Write a query to identify the top 10 most influential users, defined as those whose posts have the highest ratio of comments to views, but only considering posts made in the last 6 months """, context_user))

```sql
WITH recent_posts AS (
    SELECT post_id, user_id
    FROM posts
    WHERE post_date >= DATE('now', '-6 months')
),
post_comments AS (
    SELECT p.user_id, p.post_id, COUNT(c.comment_id) AS num_comments, p.views
    FROM recent_posts p
    JOIN comments c ON p.post_id = c.post_id
    GROUP BY p.user_id, p.post_id
),
user_influence AS (
    SELECT user_id, SUM(num_comments) AS total_comments, SUM(views) AS total_views
    FROM post_comments
    GROUP BY user_id
)
SELECT user_id, 
       CAST(total_comments AS REAL) / total_views AS comments_to_views_ratio
FROM user_influence
ORDER BY comments_to_views_ratio DESC
LIMIT 10;
```


Report SQL query from table names - Continued

I created a prompt to test the old and new prompt medthod using a complex and simple query, below is what I have learnt.

I learnt that the Open Ai - 3.5 is very good at generating queries accuratly, I did a comparision with the simple 'shall all users' query and discovered that they new and old prompt model gave similar outputs that could generate accurate results.

#new 
context_user = context.copy()
print(return_CCRMSQL("""show all users""", context_user)) 
returns =
SELECT * FROM users;    

#old
old_context_user = old_context.copy()
print(return_CCRMSQL("show all users", old_context_user))
returns =
SELECT * FROM employees;

However , the old prompt seems to halucinate while because it created employees indead of users.

Also when more complex queries where supplied in the next prompts I saw that i got different returns with different decree of efficeincy example given below 

Example : 
#old
print(return_CCRMSQL("""Assume you have a table of user posts on a forum with a posts table (columns: post_id, user_id, post_date, views) and a comments table (columns: comment_id, post_id, user_id, comment_date). Write a query to identify the top 10 most influential users, defined as those whose posts have the highest ratio of comments to views, but only considering posts made in the last 6 months """, old_context_user))

This is your SQL:

```sql
SELECT user_id, SUM(comment_count) / SUM(views) AS comment_to_views_ratio
FROM (
    SELECT p.user_id, COUNT(c.comment_id) AS comment_count, p.views
    FROM posts p
    JOIN comments c ON p.post_id = c.post_id
    WHERE p.post_date >= DATE_SUB(NOW(), INTERVAL 6 MONTH)
    GROUP BY p.post_id
) AS subquery
GROUP BY user_id
ORDER BY comment_to_views_ratio DESC
LIMIT 10;
```


#new
context_user = context.copy()
print(return_CCRMSQL("""Assume you have a table of user posts on a forum with a posts table (columns: post_id, user_id, post_date, views) and a comments table (columns: comment_id, post_id, user_id, comment_date). Write a query to identify the top 10 most influential users, defined as those whose posts have the highest ratio of comments to views, but only considering posts made in the last 6 months """, context_user))

```sql
WITH recent_posts AS (
    SELECT post_id, user_id
    FROM posts
    WHERE post_date >= DATE('now', '-6 months')
),
post_comments AS (
    SELECT p.user_id, p.post_id, COUNT(c.comment_id) AS num_comments, p.views
    FROM recent_posts p
    JOIN comments c ON p.post_id = c.post_id
    GROUP BY p.user_id, p.post_id
),
user_influence AS (
    SELECT user_id, SUM(num_comments) AS total_comments, SUM(views) AS total_views
    FROM post_comments
    GROUP BY user_id
)
SELECT user_id, 
       CAST(total_comments AS REAL) / total_views AS comments_to_views_ratio
FROM user_influence
ORDER BY comments_to_views_ratio DESC
LIMIT 10;
```
Old Prompt :
The logic is akk in one big block.
This could be hard to read and as the code get more complex.

New Prompt :
The logic is broken down into smaller named steps.
This makes the code mush easier to read and understan whihc is considered a bext practice.